In [1]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd
from surprise.model_selection import cross_validate

ПАКЕТ SURPRISE

используйте данные MovieLens 1M

можно использовать любые модели из пакета

получите RMSE на тестовом сете 0.87 и ниже

Комментарий преподавателя :
В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.

In [2]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [3]:

movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [4]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [5]:
dataset = dataset.sort_values(by=['uid'])
dataset

,uid,iid,rating
16698601,1.0,Hollow Man (2000),2.0
14745400,1.0,RoboCop 2 (1990),2.5
6846346,1.0,Weekend at Bernie's (1989),1.5
16391348,1.0,"Running Man, The (1987)",3.5
9878107,1.0,Spawn (1997),1.5
...,...,...,...
2064721,283228.0,Star Wars: Episode IV - A New Hope (1977),5.0
19741333,283228.0,Catch Me If You Can (2002),4.5
8827486,283228.0,Butch Cassidy and the Sundance Kid (1969),4.5
5893806,283228.0,"Philadelphia Story, The (1940)",4.5


In [6]:
ratings.rating.min()

0.5

In [7]:
ratings.rating.max()

5.0

In [11]:
dataset_id = dataset.sort_values(by=['iid'])
dataset_id

,uid,iid,rating
23440156,277775.0,"""Great Performances"" Cats (1998)",0.5
23440105,222307.0,"""Great Performances"" Cats (1998)",0.5
23440141,261872.0,"""Great Performances"" Cats (1998)",3.0
23440023,109040.0,"""Great Performances"" Cats (1998)",2.5
23440121,236981.0,"""Great Performances"" Cats (1998)",4.0
...,...,...,...
27186232,215605.0,貞子3D (2012),0.5
27186229,183200.0,貞子3D (2012),1.0
27186233,215651.0,貞子3D (2012),3.0
27476241,132769.0,줄탁동시 (2012),5.0


In [14]:
new_dataset_id = dataset_id[:3000000]
new_dataset_id.uid.value_counts()

123100.0    2364
117490.0     911
212343.0     790
134596.0     772
242683.0     715
            ... 
59926.0        1
274882.0       1
59928.0        1
170558.0       1
40263.0        1
Name: uid, Length: 235202, dtype: int64

In [15]:

new_dataset_id.iid.value_counts()

American Beauty (1999)                                                                      60820
Apollo 13 (1995)                                                                            58665
Back to the Future (1985)                                                                   57492
Batman (1989)                                                                               54448
Aladdin (1992)                                                                              51839
                                                                                            ...  
Basta guardarla (1970)                                                                          1
Argo (2004)                                                                                     1
Bailout at 43,000 (1957)                                                                        1
All Blossoms Again: Pedro Costa, Director (Tout refleurit: Pedro Costa, cinéaste) (2006)        1
A River of Waste: Th

In [16]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(new_dataset_id, reader)

In [24]:
trainset, testset = train_test_split(data, test_size=.05)

In [37]:

algo_2 = KNNWithMeans(k=15, sim_options={'name': 'pearson_baseline', 'user_based': False})

In [38]:

algo_2.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [39]:

test_pred = algo_2.test(testset)

In [40]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8680


0.8680371264685356

In [41]:
cross_validate(algo_2, data, measures=['RMSE'], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8835  0.8845  0.8841  0.8855  0.8834  0.8842  0.0007  
Fit time          37.96   36.37   35.47   33.89   37.26   36.19   1.42    
Test time         47.32   47.06   44.73   46.56   46.57   46.45   0.91    


{'test_rmse': array([0.88354097, 0.88450314, 0.88414148, 0.88545713, 0.88340878]),
 'fit_time': (37.95917105674744,
  36.36908030509949,
  35.465028524398804,
  33.89393877983093,
  37.261131286621094),
 'test_time': (47.3197066783905,
  47.060691833496094,
  44.72955846786499,
  46.55866289138794,
  46.57366394996643)}